In [8]:
# STEP 1: Getting the pre-trained Model

# Install necessary packages
! pip install demucs
! pip install torchaudio
! pip install os
! pip install librosa
! pip install np

from demucs.pretrained import get_model

# Load the pre-trained Demucs model
model_name = "htdemucs"  # Change to the specific model name you wish to use
separator = get_model(model_name)

# Verifying the model is loaded
print(separator)

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


   ---------------------------------------- 0.0/260.1 kB ? eta -:--:--
   ------------------ --------------------- 122.9/260.1 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 260.1/260.1 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.3 kB ? eta -:--:--
   ---------------------------------------- 75.3/75.3 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   -------- ------------------------------- 0.6/2.7 MB 17.5 MB/s eta 0:00:01
   --------------------- ------------------ 1.5/2.7 MB 18.5 MB/s eta 0:00:01
   --------------------------------- ------ 2.3/2.7 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.7 MB 15.7 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 14.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.6 kB ? eta -:--:--
   ---------------------------------------- 64.6/64.6 kB 3.4 MB/s eta 0:00:00
   ----------------

In [32]:
# STEP 2: Normalize and Preprocess the Data

import os
import librosa
import numpy as np

# Normalize and convert audio to spectrogram functions
def normalize_audio(audio, target_sr=22050):
    audio_resampled = librosa.resample(audio, orig_sr=librosa.get_samplerate(audio), target_sr=target_sr)
    return audio_resampled

def audio_to_spectrogram(audio, sr=22050, n_fft=2048, hop_length=512):
    # Compute the Short-Time Fourier Transform
    stft = librosa.stft(audio, n_fft=n_fft, hop_length=hop_length)
    # Convert the magnitude to decibels
    spectrogram = librosa.amplitude_to_db(np.abs(stft))
    return spectrogram

# Helper function to process and save the spectrogram
def process_and_save(audio_path, save_spect_dir, target_sr=22050):
    audio, _ = librosa.load(audio_path, sr=None)
    audio_norm = normalize_audio(audio, target_sr)
    spectrogram = audio_to_spectrogram(audio_norm, sr=target_sr)
    
    # Create the directory if it doesn't exist
    os.makedirs(save_spect_dir, exist_ok=True)
    
    # Save the spectrogram as a numpy file
    spectrogram_path = os.path.join(save_spect_dir, os.path.basename(audio_path).replace('.wav', '.npy'))
    np.save(spectrogram_path, spectrogram)
    print(f'Saved spectrogram: {spectrogram_path}')

save_spect_path = './outputs/spectrograms_musdb18'
musdb18_path = './musdb18'

# Define the base path and output path using absolute paths
base_path = os.path.abspath('./musdb18')
save_spect_path = os.path.abspath('./outputs/spectrograms_musdb18')

print(f"Base Path: {base_path}")
print(f"Save Spectrogram Path: {save_spect_path}")

path = 'c:\Users\mikea\OneDrive\Documents\GitHub\gen-ai-mp3-to-musescore\python\musdb18'

print 
with disable_file_system_redirection():
    print os.listdir(path)
print os.listdir(path)

Base Path: c:\Users\mikea\OneDrive\Documents\GitHub\gen-ai-mp3-to-musescore\python\musdb18
Save Spectrogram Path: c:\Users\mikea\OneDrive\Documents\GitHub\gen-ai-mp3-to-musescore\python\outputs\spectrograms_musdb18


In [33]:
# Process each song in the dataset
for subset in ['train', 'test']:
    subset_dir = os.path.join(base_path, subset)
    save_subset_dir = os.path.join(save_spect_path, subset)

    print(f"Processing subset: {subset_dir}")

    if not os.path.isdir(subset_dir):
        print(f"Directory not found: {subset_dir}")
        continue

    for song in os.listdir(subset_dir):
        song_dir = os.path.join(subset_dir, song)

        print(f"Song entry found: {song}: full path: {song_dir}")

        if os.path.isdir(song_dir):
            print(f"Confirmed directory: {song_dir}")

            mixture_path = os.path.join(song_dir, 'mixture.wav')

            print(f"Checking mixture path: {mixture_path}")

            if os.path.exists(mixture_path):
                process_and_save(mixture_path, save_subset_dir)
            else:
                print(f"Mixture file does not exist: {mixture_path}")
        else:
            print(f"Entry is not a directory: {song_dir}")

print("Preprocessing complete.")

Processing subset: c:\Users\mikea\OneDrive\Documents\GitHub\gen-ai-mp3-to-musescore\python\musdb18\train
Directory not found: c:\Users\mikea\OneDrive\Documents\GitHub\gen-ai-mp3-to-musescore\python\musdb18\train
Processing subset: c:\Users\mikea\OneDrive\Documents\GitHub\gen-ai-mp3-to-musescore\python\musdb18\test
Directory not found: c:\Users\mikea\OneDrive\Documents\GitHub\gen-ai-mp3-to-musescore\python\musdb18\test
Preprocessing complete.
